In [1]:
fec_min = "2000-01-01"
fec_max = "2025-02-01"

# Configuración spark

In [2]:
path_datos = "gs://bucket-tfm-llc/datos"

# Librerias

In [3]:
pip install alpha_vantage

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark.sql.functions as F
from alpha_vantage.timeseries import TimeSeries


In [5]:
api_key = "D1KXHTVQ2A275VB0"

# Carga de datos

In [6]:
# Hacemos la lectura de los datos financieros a partir de la api
ts = TimeSeries(key=api_key, output_format='pandas')
data, meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [7]:
print(data.head())

            1. open   2. high  3. low  4. close   5. volume
date                                                       
2025-06-06  212.400  213.8699  210.50    213.57  39832500.0
2025-06-05  209.550  212.8100  207.56    207.91  51979243.0
2025-06-04  206.550  208.1800  205.18    207.23  29915592.0
2025-06-03  207.105  208.9469  205.03    205.71  33139121.0
2025-06-02  204.980  207.0000  202.68    206.65  29113319.0


In [8]:
# Nos quedamos con los datos entre los años definidos
data_fil = data[(data.index >= fec_min) & (data.index <= fec_max)]
data_fil.columns = [var.split(". ")[1] for var in data.columns]

In [9]:
# Pasamos los datos a un spark dataframe
df_data = spark.createDataFrame(data_fil.reset_index())

In [10]:
df_data.count()

6309

In [11]:
df_data.summary().show()

+-------+-----------------+-----------------+-----------------+-----------------+--------------------+
|summary|             open|             high|              low|            close|              volume|
+-------+-----------------+-----------------+-----------------+-----------------+--------------------+
|  count|             6309|             6309|             6309|             6309|                6309|
|   mean|593.3223745760007|600.0108554446048|585.8828748930082|593.0824456966238|1.1306734247424314E7|
| stddev|927.8425149015573| 938.137159963499|916.1059603175174|927.0595077718248|1.7253855835222587E7|
|    min|             5.91|              6.1|             5.51|             5.97|            881337.0|
|    25%|            47.22|             48.2|            46.06|            47.22|           3648791.0|
|    50%|            165.5|           168.47|         162.8701|            165.8|           5625700.0|
|    75%|           637.14|           645.93|           627.35|          

In [12]:
df_data.orderBy("date", ascending=False).show(5)

+-------------------+-------+------+------+------+-----------+
|               date|   open|  high|   low| close|     volume|
+-------------------+-------+------+------+------+-----------+
|2025-01-31 00:00:00|  236.5|240.29|236.41|237.68|3.6162377E7|
|2025-01-30 00:00:00| 237.14|237.95|232.22|234.64|3.2020728E7|
|2025-01-29 00:00:00|239.015|240.39|236.15|237.07|2.6091716E7|
|2025-01-28 00:00:00| 234.29|241.77|233.98|238.15|4.1587188E7|
|2025-01-27 00:00:00| 226.21|235.61|225.86|235.42|4.9428332E7|
+-------------------+-------+------+------+------+-----------+
only showing top 5 rows



# Output

In [13]:
# Guardamos los datos brutos para tratarlos más adelante
df_data.write.mode("overwrite").format("parquet").save(f"{path_datos}/datos_financieros_amzn_raw")